Code below will visit the websites to check if they are reachable or active.

In [ ]:
import csv
import requests

# Initialize counters
visited_count = 0
valid_count = 0
not_valid_count = 0
error_count = 0

# Initialize empty lists
X = []
y = []

# Open CSV file
with open('/content/drive/MyDrive/FYP_Testing&WhatNot/website list - Sheet1-5 - website list - Sheet1-5.csv', newline='') as f:
    reader = csv.reader(f)
    next(reader) # Skip header row

    # Loop over each row in CSV
    for row in reader:
        url = row[0]
        label = row[1]
        try:
            # Send request to website
            response = requests.get(url)

            # Check if website is valid
            if response.status_code == 200:
                valid_count += 1
                X.append(url)
                y.append(label)
            else:
                not_valid_count += 1
        except:
            error_count += 1

        visited_count += 1

# Print counters
print("Visited websites:", visited_count)
print("Valid websites:", valid_count)
print("Not valid websites:", not_valid_count)
print("Errors:", error_count)


Visited websites: 164
Valid websites: 113
Not valid websites: 39
Errors: 12


Code below checks if the website is active and valid.

In [ ]:
import csv
import requests

url_list = []

with open('/content/drive/MyDrive/FYP_Testing&WhatNot/website list - Sheet1-5 - website list - Sheet1-5.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # skip header row
    for row in reader:
        url = row[0]
        url_list.append(url)

for url in url_list:
    try:
        r = requests.get(url)
        if r.status_code == 200:
            print(url, "is a valid website")
        else:
            print(url, "is not a valid website")
    except:
        print(url, "is not a valid website")


https://irit-therapy.com/wp-admin/ent.html is a valid website
https://bfj.pages.dev/ is a valid website
https://google.com is a valid website
https://bxc.pages.dev/ is a valid website
http://facebook.com is a valid website
http://reddit.com is a valid website
https://wwxhajudjgwjklckvzgs6.web.app/ is a valid website
http://site9548676.92.webydo.com/ is a valid website
https://wwxhajudjgwjklckvzgs1.web.app/ is a valid website
https://cf-ipfs.com/ipfs/QmchiPZKHT5fSC9DRvMz3aNsBSMLANbM1fNmCBk5kpszsx is a valid website
https://cloudflare-ipfs.com/ipfs/bafybeigvnd42jom7e3wimgpo2abup6bsnp53cbes2gkjsp4nuyidanryv4 is a valid website
https://www.mmu.edu.my/ is a valid website
https://www.digitalocean.com/ is a valid website
https://realpython.com/ is a valid website
https://www.kdnuggets.com/ is a valid website
https://homeapks.com/wp-content/WesBav2/ is a valid website
http://sertimadioprtugachurigiencocert.clinicauruguai.com/ is a valid website
https://www.access-jaccs.eakqkkw.cn/Logins/mylogi

Code below does the mounting process on Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Code below will get the features from the websites in the "website list - Sheet1-5 - website list - Sheet1-5.csv" file and output the features into the "elijah_features.csv" file.

The features that are being extracted will be the url (duh), some keywords that are consistent among phishing websites ["login", "password", "verify", "account", "security", "banking", "credit card"], length of the JavaScript code and the specific JavaScript code that are usually found with phishing websites.

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

# Define a function to extract phishing site features
def extract_features(url):
    try:
        start_time = time.time()
        response = requests.get(url, timeout=5)  # Set timeout to 5 seconds

        # Check if the process exceeds the time limit
        elapsed_time = time.time() - start_time
        if elapsed_time > 300:  # Stop gathering data after 5 minutes (300 seconds)
            return None

        soup = BeautifulSoup(response.text, 'html.parser')
        scripts = soup.find_all('script')

        # Check if there are scripts to extract features from
        if len(scripts) == 0:
            return None

        script_content = [script.text for script in scripts]
        script_string = "\n".join(script_content)

        features = []
        # Add text detection feature
        keywords = ["login", "password", "verify", "account", "security", "banking", "credit card"]
        for keyword in keywords:
            features.append(int(keyword in response.text.lower()))

        # Add JavaScript features
        features.append(len(script_string))
        features.append(len(scripts))
        features.append(len(re.findall(r"\bfunction\b", script_string)))
        features.append(len(re.findall(r"\beval\b", script_string)))
        features.append(len(re.findall(r"\bwindow\b", script_string)))
        features.append(len(re.findall(r"\bsetTimeout\b", script_string)))
        features.append(len(re.findall(r"\bsetInterval\b", script_string)))
        features.append(len(re.findall(r"\bexec\b", script_string)))
        features.append(len(re.findall(r"\bcookie\b", script_string)))
        features.append(len(re.findall(r"\blocation\b", script_string)))
        features.append(len(re.findall(r"\bXMLHttpRequest\b", script_string)))
        features.append(len(re.findall(r"\biframe\b", script_string)))
        features.append(len(re.findall(r"\bdocument\.cookie\b", script_string)))
        features.append(len(re.findall(r"\bunescape\b", script_string)))
        features.append(len(re.findall(r"\bfromCharCode\b", script_string)))

        return features
    except:
        return None

# Load the CSV file with original data
original_df = pd.read_csv("/content/drive/MyDrive/FYP_Testing&WhatNot/website list - Sheet1-5 - website list - Sheet1-5.csv")

# Create an empty list to store the rows
rows = []

# Loop through each URL in the original data
for i, row in original_df.iterrows():
    url = row["url"]
    label = row["label"]

    # Extract features for the URL
    features = extract_features(url)

    # Check if the process exceeded the time limit or no features were extracted
    if features is None:
        continue

    # Create a dictionary representing a row in the CSV file
    row_dict = {
        "url": url,
        "label": label,
        "text_detection": features[0],
        "script_length": features[1],
        "num_scripts": features[2],
        "num_functions": features[3],
        "num_eval": features[4],
        "num_window": features[5],
        "num_setTimeout": features[6],
        "num_setInterval": features[7],
        "num_exec": features[8],
        "num_cookie": features[9],
        "num_location": features[10],
        "num_XMLHttpRequest": features[11],
        "num_iframe": features[12],
        "num_document_cookie": features[13],
        "num_unescape": features[14],
        "num_fromCharCode": features[15]
    }

    # Append the row dictionary to the list of rows
    rows.append(row_dict)

# Convert the list of rows to a DataFrame
feature_df = pd.DataFrame(rows)

# Save the extracted features to a new CSV file
feature_df.to_csv("/content/drive/MyDrive/FYP_Testing&WhatNot/elijah_features.csv", index=False)



<ipython-input-5-282c4ed0bcd1>:18: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.text, 'html.parser')


Training a model with the Random Forest Classifier - this is a classification problem after all.

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the CSV file with extracted features and labels
data_df = pd.read_csv("/content/drive/MyDrive/FYP_Testing&WhatNot/elijah_features.csv")

# Separate features (X) and labels (y)
X = data_df.drop(["label", "url"], axis=1)  # Remove the "label" and "url" columns from X
y = data_df["label"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier()

# Train the classifier on the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)



Accuracy: 0.8928571428571429


Running the training model on the collected features, can tell that there are a lot of false positives.

In [21]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import joblib
from sklearn.metrics import classification_report

# Load the extracted features from the CSV file
data = pd.read_csv('/content/drive/MyDrive/FYP_Testing&WhatNot/elijah_features.csv')

# Define the desired features to keep
desired_features = ['script_length', 'text_detection','num_scripts', 'num_functions', 'num_eval', 'num_window', 'num_setTimeout',
                    'num_setInterval', 'num_exec', 'num_cookie', 'num_location', 'num_XMLHttpRequest', 'num_iframe',
                    'num_document_cookie', 'num_unescape', 'num_fromCharCode']

# Filter the features to keep only the desired ones
X = data[desired_features]
y = data['label']

# Load the pre-trained model from the joblib file
clf = joblib.load('/content/drive/MyDrive/FYP_Testing&WhatNot/model_elijah.joblib')

# Load the CSV file with websites and labels
website_data = pd.read_csv('/content/drive/MyDrive/FYP_Testing&WhatNot/elijah_features.csv')

# Iterate over the websites in the CSV file
for i, row in website_data.iterrows():
    website_url = row['url']
    label = row['label']

    try:
        # Send a GET request to the website URL
        response = requests.get(website_url)

        # Check for HTTP errors
        response.raise_for_status()

    except requests.HTTPError:
        print(f"Website {website_url} is unreachable")
        continue

    except requests.RequestException:
        print(f"An error occurred while accessing the website {website_url}")
        continue

    # Extract features from the website JavaScript content
    js_content = []
    soup = BeautifulSoup(response.content, 'html.parser')
    scripts = soup.find_all('script')

    if len(scripts) == 0:
        print(f"Status of website {website_url} is unknown due to the absence of scripts to filter!")
        continue

    for script in scripts:
        js_content.append(script.string)

    # Extract the desired features from the JavaScript content
    js_features = []
    for feature_name in X.columns:
        feature_count = sum(script.count(feature_name) for script in js_content if script is not None)
        js_features.append(feature_count)

    # Convert the features into a numpy array
    website_features = np.array(js_features).reshape(1, -1)

    if website_features.size == 0:
        print(f"Status of website {website_url} is unknown due to the absence of features!")
        continue

    # Make a prediction using the trained model
    prediction_prob = clf.predict_proba(website_features)
    threshold = 0.45  # Adjust the threshold as needed
    if prediction_prob[0][1] >= threshold:
        prediction = 'phishing'
    else:
        prediction = 'safe'

    # Display the prediction result and compare with the label
    print(f"Website: {website_url}")
    print(f"Prediction: {prediction}")
    print(f"Label: {label}")
    print()



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Website: https://irit-therapy.com/wp-admin/ent.html
Prediction: safe
Label: phishing

Website: https://bfj.pages.dev/
Prediction: safe
Label: phishing



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Website: https://bxc.pages.dev/
Prediction: safe
Label: phishing

Website: https://wwxhajudjgwjklckvzgs6.web.app/
Prediction: safe
Label: phishing



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Website: https://www.techcrunch.com
Prediction: safe
Label: safe



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Website: https://www.engadget.com
Prediction: safe
Label: safe

Website: https://www.lifehacker.com
Prediction: safe
Label: safe



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


I was testing the prediction model with random websites from the internet and it still comes up as a false positive ):

Sometimes the prediction model also comes up with a "scripts cannot be extracted" error to break up the monotony xD

In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import joblib

# Prompt the user to enter a website URL
website_url = input("Enter a website URL: ")

# Extract features from the website JavaScript content
js_content = []
response = requests.get(website_url)
soup = BeautifulSoup(response.content, 'html.parser')
scripts = soup.find_all('script')

if len(scripts) == 0:
    print("Status unknown due to the absence of scripts to filter!")
    exit()

for script in scripts:
    js_content.append(script.string)

# Extract the desired features from the JavaScript content
js_features = []
for feature_name in X.columns:
    feature_count = sum(script.count(feature_name) for script in js_content if script is not None)
    js_features.append(feature_count)

# Convert the features into a numpy array
website_features = np.array(js_features).reshape(1, -1)

if website_features.size == 0:
    print("Status unknown due to the absence of features!")
    exit()

# Load the trained model
model = joblib.load('trained_model.joblib')

# Make a prediction using the trained model
prediction = model.predict(website_features)

# Display the prediction result
if prediction[0] == 1:
    print("The website is classified as phishing.")
else:
    print("The website is classified as safe.")


I got frustrated and trained the model again, this time with some specifics - stuff like weighted average, precision, recall, f1-score and support.

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/FYP_Testing&WhatNot/elijah_features.csv')

# Split the dataset into features (X) and labels (y)
X = data.drop(['url', 'label'], axis=1)
y = data['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the Random Forest Classifier
clf = RandomForestClassifier()

# Train the model
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

# Save the trained model
joblib.dump(clf, '/content/drive/MyDrive/FYP_Testing&WhatNot/model_elijah.joblib')


              precision    recall  f1-score   support

    phishing       0.88      0.78      0.82         9
        safe       0.90      0.95      0.92        19

    accuracy                           0.89        28
   macro avg       0.89      0.86      0.87        28
weighted avg       0.89      0.89      0.89        28



['/content/drive/MyDrive/FYP_Testing&WhatNot/model_elijah.joblib']

My first go at training the model, it only takes in the JavaScript features. Keep it a stack with you I'm going to change the features, it might be the features causing the false positives.

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse
import pandas as pd
import ssl
from datetime import datetime

# Disable SSL certificate verification
ssl._create_default_https_context = ssl._create_unverified_context

# Define a function to extract JavaScript features from a website
def extract_js_features(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        scripts = soup.find_all('script')

        # Check if there are scripts to extract features from
        if len(scripts) == 0:
            print("No scripts found on the website:", url)
            return None

        script_content = [script.text for script in scripts]
        script_string = "\n".join(script_content)

        # Extract the URL length
        url_length = len(urlparse(url).path)

        features = []
        # Extract JavaScript features
        features.append(len(script_string))
        features.append(len(scripts))
        features.append(len(re.findall(r"\bfunction\b", script_string)))
        features.append(len(re.findall(r"\beval\b", script_string)))
        features.append(len(re.findall(r"\bwindow\b", script_string)))
        features.append(len(re.findall(r"\bsetTimeout\b", script_string)))
        features.append(len(re.findall(r"\bsetInterval\b", script_string)))
        features.append(len(re.findall(r"\bexec\b", script_string)))
        features.append(len(re.findall(r"\bcookie\b", script_string)))
        features.append(len(re.findall(r"\blocation\b", script_string)))
        features.append(len(re.findall(r"\bXMLHttpRequest\b", script_string)))
        features.append(len(re.findall(r"\biframe\b", script_string)))
        features.append(len(re.findall(r"\bdocument\.cookie\b", script_string)))
        features.append(len(re.findall(r"\bunescape\b", script_string)))
        features.append(len(re.findall(r"\bfromCharCode\b", script_string)))

        # Replace the domain age feature with the URL length
        features.append(url_length)

        # Check SSL certificate information
        ssl_info = ssl.get_server_certificate((urlparse(url).hostname, 443))
        x509 = ssl.load_certificate(ssl.PEM, ssl_info)
        cert_info = x509.get_subject()
        expiry_date = datetime.strptime(cert_info.get_notAfter().decode('ascii'), "%Y%m%d%H%M%SZ")
        days_until_expiry = (expiry_date - datetime.now()).days
        features.append(days_until_expiry)

        return features
    except:
        return None

# Load the CSV file with original data
original_df = pd.read_csv("/content/drive/MyDrive/FYP_Testing&WhatNot/website list - Sheet1-5 - website list - Sheet1-5.csv")

# Create an empty DataFrame to store the extracted features
feature_df = pd.DataFrame(columns=["url", "script_length", "num_scripts", "num_functions", "num_eval", "num_window", "num_setTimeout", "num_setInterval", "num_exec", "num_cookie", "num_location", "num_XMLHttpRequest", "num_iframe", "num_document_cookie", "num_unescape", "num_fromCharCode", "url_length", "days_until_expiry"])

# Loop through each website and extract features
for url in original_df["url"]:
    features = extract_js_features(url)
    if features:
        feature_df = feature_df.append(pd.Series([url] + features, index=feature_df.columns), ignore_index=True)

# Merge the extracted features DataFrame with the original DataFrame on the "url" column
merged_df = pd.merge(original_df, feature_df, on="url")

# Save the merged DataFrame to a CSV file
merged_df.to_csv("/content/drive/MyDrive/FYP_Testing&WhatNot/javascript_features.csv", index=False)



No scripts found on the website: http://site9548676.92.webydo.com/
No scripts found on the website: https://homeapks.com/wp-content/WesBav2/
No scripts found on the website: https://www.tesdi.mx/pack/
No scripts found on the website: https://gateway.ipfs.io/ipfs/bafybeic6sjz65ptdmfzxsj4nfboyyu4c7t5kwz5uwqh6tr7rfkdhw737dm/g46h-0o-0dol3-05gkj-95ghw-enfh-934ufhf.html
No scripts found on the website: https://ipfs.eth.aragon.network/ipfs/bafybeif65xcjv5hq2lpmhjjhaufqkx3hps5vjd5entcixn46ilofmsvs4i/roundcube


<ipython-input-6-4777ccff35d3>:16: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.text, 'html.parser')


No scripts found on the website: https://cloudflare-ipfs.com/ipfs/bafybeif65xcjv5hq2lpmhjjhaufqkx3hps5vjd5entcixn46ilofmsvs4i/others
No scripts found on the website: https://53467michsafeul345667.web.app/
No scripts found on the website: https://53467michsafeul345667.firebaseapp.com/
No scripts found on the website: http://breton-gille.com/hb/
No scripts found on the website: http://breton-gille.com/hb
No scripts found on the website: http://racingallesand.wpenginepowered.com/
No scripts found on the website: http://mercedesbenzsl.wpenginepowered.com/
No scripts found on the website: https://www.microsoft.com/
No scripts found on the website: http://enzoferrari356.wpenginepowered.com/
No scripts found on the website: https://ugde.magichecoccole.com/de/ugde/79fVEv8XxeVx7491/login.php?Nk65L7l2kcar75cI7f00eJ7ybyI0rpQh0w16x7P1qFSQQtcmdZKCZFzJikQ1gZHRi3wQtl9l6nCHOsxDcDw6dTrguz7tF7mixwY8hgZqWPRC27sSrMwiBKeA#781e5e245d69b566979b86e28d23f2c7%23781e5e245d69b566979b86e28d23f2c7%23781e5e245d69b56